#### Library Imports



In [43]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE

#### Loading Data

In [44]:
job_postings = pd.read_csv('data/job_postings.csv')
job_skills = pd.read_csv('data/job_details/job_skills.csv')

companies = pd.read_csv('data/company_details/companies.csv')
mapping_skills = pd.read_csv('data/maps/skills.csv')
employee_count = pd.read_csv('data/company_details/employee_counts.csv')

#### Dataset summary

In [45]:
unique_values = {
    'job_postings': job_postings['job_id'].nunique(), # details of job postings
    'job_skills': job_skills['job_id'].nunique(), # job posting with the required skill abbreviated
    'companies': companies['company_id'].nunique(), # information on the company
    'skills': mapping_skills['skill_name'].nunique() # maps between skill abbreviations and full skill names
    }
unique_values

{'job_postings': 33246, 'job_skills': 32422, 'companies': 11361, 'skills': 35}

#### Handling the integration of job_skills with mapping_skills
Merging job_skills with mapping_skills

In [46]:
# Merging job_skills with mapping_skills to replace skill_abr with skill_name
merged_skills = pd.merge(job_skills, mapping_skills, on='skill_abr', how='left')
merged_skills = merged_skills.drop('skill_abr', axis=1)
merged_skills.head()

,job_id,skill_name
0,3690843087,Accounting/Auditing
1,3690843087,Finance
2,3691763971,Management
3,3691763971,Manufacturing
4,3691775263,Management


Aggregating Skills by Job ID

In [47]:
# Aggregation process is done to handle duplicate 'job_id's resulting from multiple skills per job:

# Grouping by 'job_id' and concatenating 'skill_name' values
grouped_skills = merged_skills.groupby('job_id')['skill_name'].agg(lambda x: ', '.join(x)).reset_index()

# Display the resulting DataFrame with unique 'job_id' rows and concatenated skills
print(grouped_skills)

           job_id                                    skill_name
0         3958427  Design, Art/Creative, Information Technology
1        85008768                   Sales, Business Development
2       102339515                   Business Development, Sales
3       108965123                                Administrative
4       133114754                   Sales, Business Development
...           ...                                           ...
32417  3757937095                        Information Technology
32418  3757938018                     Management, Manufacturing
32419  3757938019                                   Engineering
32420  3757940025                     Management, Manufacturing
32421  3757940104                                         Other

[32422 rows x 2 columns]


#### Handling the integration of job_postings with grouped_skills
Checking for missing data in Company ID's

In [48]:
missing_id = job_postings['company_id'].isnull().sum()
missing_id

654

Dropping rows with empty company_id's

In [49]:
# Shape of dataset before removing missing values
job_postings.shape

(33246, 28)

In [50]:
job_postings.dropna(subset=['company_id'], inplace=True)
job_postings.shape

(32592, 28)

Merging job_postings with grouped_skills

In [51]:
merged_job_postings_skills = pd.merge(job_postings, grouped_skills, on='job_id', how='left')

Converting company data type to int from float

In [52]:
# Because there used to be missing values in company id's, NaN was considered as float value, therefore all data in the column was converted
# to float from int
if 'company_id' in merged_job_postings_skills.columns:
    merged_job_postings_skills['company_id'] = merged_job_postings_skills['company_id'].astype('Int64')

In [53]:
merged_job_postings_skills.head()

,job_id,company_id,title,description,max_salary,med_salary,min_salary,pay_period,formatted_work_type,location,...,formatted_experience_level,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,scraped,skill_name
0,3757940104,553718,Hearing Care Provider,Overview\n\nHearingLife is a national hearing ...,NaN,5250.00,NaN,MONTHLY,Full-time,"Little River, SC",...,Entry level,NaN,1.699090e+12,careers-demant.icims.com,0,FULL_TIME,USD,BASE_SALARY,1699138101,Other
1,3757940025,2192142,Shipping & Receiving Associate 2nd shift (Beav...,Metalcraft of Mayville\nMetalcraft of Mayville...,NaN,NaN,NaN,NaN,Full-time,"Beaver Dam, WI",...,NaN,NaN,1.699080e+12,www.click2apply.net,0,FULL_TIME,NaN,NaN,1699085420,"Management, Manufacturing"
2,3757938019,474443,"Manager, Engineering",\nThe TSUBAKI name is synonymous with excellen...,NaN,NaN,NaN,NaN,Full-time,"Bessemer, AL",...,NaN,Bachelor's Degree in Mechanical Engineering pr...,1.699080e+12,www.click2apply.net,0,FULL_TIME,NaN,NaN,1699085644,Engineering
3,3757938018,18213359,Cook,descriptionTitle\n\n Looking for a great oppor...,NaN,22.27,NaN,HOURLY,Full-time,"Aliso Viejo, CA",...,Entry level,NaN,1.699080e+12,jobs.apploi.com,0,FULL_TIME,USD,BASE_SALARY,1699087461,"Management, Manufacturing"
4,3757937095,437225,Principal Cloud Security Architect (Remote),"Job Summary\nAt iHerb, we are on a mission to ...",275834.0,NaN,205956.0,YEARLY,Full-time,United States,...,Mid-Senior level,NaN,1.699090e+12,careers.iherb.com,0,FULL_TIME,USD,BASE_SALARY,1699085346,Information Technology


Reducing dimensionality of merged_job_postings_skills by dropping unnecessary columns

In [54]:
columns_to_drop = ['description', 'max_salary','med_salary','min_salary','location',
                   'pay_period', 'applies', 'formatted_work_type', 'remote_allowed',
                   'job_posting_url','views','title','application_url','application_type',
                   'expiry','skills_desc','posting_domain','sponsored','original_listed_time',
                   'work_type','currency','compensation_type','scraped','closed_time','listed_time'
                   ]
merged_job_postings_skills= merged_job_postings_skills.drop(columns=columns_to_drop,axis=1)
merged_job_postings_skills.head()


,job_id,company_id,formatted_experience_level,skill_name
0,3757940104,553718,Entry level,Other
1,3757940025,2192142,NaN,"Management, Manufacturing"
2,3757938019,474443,NaN,Engineering
3,3757938018,18213359,Entry level,"Management, Manufacturing"
4,3757937095,437225,Mid-Senior level,Information Technology


#### Handling the integration of merged_job_postings_skills with companies
Merging merged_job_postings_skills with companies


In [55]:
merged_data= pd.merge(merged_job_postings_skills, companies, on='company_id', how='left')
merged_data.head()

,job_id,company_id,formatted_experience_level,skill_name,name,description,company_size,state,country,city,zip_code,address,url
0,3757940104,553718,Entry level,Other,HearingLife,HearingLife is a national hearing care company...,5.0,New Jersey,US,Somerset,08873,580 Howard Avenue,https://www.linkedin.com/company/hearing-life
1,3757940025,2192142,NaN,"Management, Manufacturing","Metalcraft of Mayville, Inc.","Headquartered in Mayville, Wisconsin, Metalcra...",4.0,WI,US,Mayville,53050,1000 Metalcraft Drive,https://www.linkedin.com/company/metalcraft-of...
2,3757938019,474443,NaN,Engineering,"U.S. Tsubaki Power Transmission, LLC","U.S. Tsubaki Power Transmission, LLC is a subs...",4.0,Illinois,US,Wheeling,60090,301E Marquardt Drive,https://www.linkedin.com/company/u.s.-tsubaki-...
3,3757938018,18213359,Entry level,"Management, Manufacturing",Episcopal Communities & Services,Episcopal Communities & Services (ECS) has bec...,4.0,California,US,Altadena,91001,2212 El Molino Ave,https://www.linkedin.com/company/episcopal-com...
4,3757937095,437225,Mid-Senior level,Information Technology,"iHerb, LLC",iHerb is on a mission to make health and welln...,5.0,California,US,Irvine,92618,17400 Laguna Canyon Rd,https://www.linkedin.com/company/iherb


Reducing dimensionality by dropping unnecessary columns

In [56]:
columns_to_drop = ['description','company_size','zip_code','url','address']
merged_data= merged_data.drop(columns=columns_to_drop,axis=1)
merged_data.head()

,job_id,company_id,formatted_experience_level,skill_name,name,state,country,city
0,3757940104,553718,Entry level,Other,HearingLife,New Jersey,US,Somerset
1,3757940025,2192142,NaN,"Management, Manufacturing","Metalcraft of Mayville, Inc.",WI,US,Mayville
2,3757938019,474443,NaN,Engineering,"U.S. Tsubaki Power Transmission, LLC",Illinois,US,Wheeling
3,3757938018,18213359,Entry level,"Management, Manufacturing",Episcopal Communities & Services,California,US,Altadena
4,3757937095,437225,Mid-Senior level,Information Technology,"iHerb, LLC",California,US,Irvine


Displaying data in a better order

In [57]:
column_order = ['job_id','company_id','name','state','country','city','skill_name','formatted_experience_level']
merged_data = merged_data[column_order]
merged_data.head()

,job_id,company_id,name,state,country,city,skill_name,formatted_experience_level
0,3757940104,553718,HearingLife,New Jersey,US,Somerset,Other,Entry level
1,3757940025,2192142,"Metalcraft of Mayville, Inc.",WI,US,Mayville,"Management, Manufacturing",NaN
2,3757938019,474443,"U.S. Tsubaki Power Transmission, LLC",Illinois,US,Wheeling,Engineering,NaN
3,3757938018,18213359,Episcopal Communities & Services,California,US,Altadena,"Management, Manufacturing",Entry level
4,3757937095,437225,"iHerb, LLC",California,US,Irvine,Information Technology,Mid-Senior level


### Handling merge with Employee Count

In [58]:
df_sorted = employee_count.sort_values(by=['company_id', 'time_recorded'], ascending=[True, False])

# Drop duplicates keeping the first occurrence, which is the latest due to the sorting.
df_unique = df_sorted.drop_duplicates(subset='company_id', keep='first')
df_unique.head()

,company_id,employee_count,follower_count,time_recorded
6221,1009,308001,15467710,1.698964e+09
10890,1016,54639,2065363,1.699063e+09
11177,1021,26591,2318150,1.699066e+09
14237,1025,71428,3540445,1.699140e+09
13438,1028,199479,9099428,1.699134e+09


In [59]:
merged_data = merged_data.merge(df_unique, on='company_id', how='left')
columns_to_drop = ['time_recorded']
merged_data= merged_data.drop(columns=columns_to_drop,axis=1)
merged_data.head()


,job_id,company_id,name,state,country,city,skill_name,formatted_experience_level,employee_count,follower_count
0,3757940104,553718,HearingLife,New Jersey,US,Somerset,Other,Entry level,1171.0,11417.0
1,3757940025,2192142,"Metalcraft of Mayville, Inc.",WI,US,Mayville,"Management, Manufacturing",NaN,300.0,2923.0
2,3757938019,474443,"U.S. Tsubaki Power Transmission, LLC",Illinois,US,Wheeling,Engineering,NaN,314.0,8487.0
3,3757938018,18213359,Episcopal Communities & Services,California,US,Altadena,"Management, Manufacturing",Entry level,36.0,305.0
4,3757937095,437225,"iHerb, LLC",California,US,Irvine,Information Technology,Mid-Senior level,1260.0,51651.0


#### Identify remaining missing data in the merged dataset
Display columns with significant missing data

In [60]:
missing_data = merged_data.isnull().sum()
significant_missing_columns = missing_data[missing_data > 0].sort_values(ascending=True)
significant_missing_columns

employee_count                   2
follower_count                   2
country                         47
city                            48
name                            51
state                           58
skill_name                    1023
formatted_experience_level    8622
dtype: int64

Handling missing values of 'name' column

In [61]:
# After merging, 51 rows of company references in job_postings were non existent in the company id's dataset. 
missing_names = merged_data['name'].isnull().sum()

null_name_indices = merged_data[merged_data['name'].isnull()]['company_id']
print("Number of missing ID's:", missing_names)
print(null_name_indices)

Number of missing ID's: 51
7305     76999667
15698     3641332
15706     3641332
15968     3641332
16405     3641332
20379      165957
20382     2313067
20384     3079381
20385      272676
20387    64734122
20389    27116461
20391     4316275
20392     1124883
20393    19115854
20394     1485063
25366        2815
25369     3514329
25380     9215353
25381    26489605
25382    79378951
25388    34771768
32290    91187899
32294     9516195
32296    89908682
32297    18872958
32298       35602
32300    18630069
32302        3657
32304    14615655
32305       88684
32306    10563070
32307      371180
32308     2902815
32309    90633414
32310    20338460
32311     2899710
32312      145145
32314       90844
32315     1434753
32316     7573454
32317     4781041
32318      718651
32319    86746333
32320    82296828
32321    82684341
32322    96649998
32323     2641066
32355     6049228
32525    10033339
32536    81149246
32580    16265700
Name: company_id, dtype: Int64


In [62]:
merged_data.to_csv('data_preprocessed/merged_data.csv', index=False)

Dropping columns where name is null

In [63]:
# 'name' of companies that did not exist in companies.csv, but did in job_postings.csv, are null
merged_data.dropna(subset=['name'], inplace=True)

Handling missing values of 'skill_name' column

In [64]:
# Handling missing values of 'skill_name' column
col_skill_fill_other = ['skill_name']
for col in col_skill_fill_other:
    merged_data[col].fillna("Other", inplace=True)

Handling missing values of 'formatted_experience_level' column

In [65]:
# Categorical columns with missing values are filled with "Not Specified"
col_experience_fill_not_specified = ['formatted_experience_level']
for col in col_experience_fill_not_specified:
    merged_data[col].fillna("Not Specified", inplace=True)


Remove null values in state, city, employee_count and follower_count

In [66]:
merged_data = merged_data.dropna(subset=['state','city','employee_count','follower_count'])

In [67]:
merged_data = merged_data[merged_data['state'] != '0']
merged_data = merged_data[merged_data['city'] != '0']
merged_data = merged_data[merged_data['country'] != '0']

Check for remaining missing values

In [68]:
remaining_missing = merged_data.isnull().sum()
remaining_missing_cols = remaining_missing[remaining_missing > 0].sort_values(ascending=False)
remaining_missing_cols

Series([], dtype: int64)

#### Categorizing experience level

In [69]:
# Data Transformation
experience_levels = merged_data['formatted_experience_level'].unique()

for value in experience_levels:
    print(value)


Entry level
Not Specified
Mid-Senior level
Director
Associate
Executive
Internship


Mapping experience to their numeric new values

In [70]:
experience_mapping = {
    'Internship': 0,
    'Entry level': 1,
    'Associate':2, 
    'Mid-Senior level': 3,
    'Director': 4,
    'Executive': 5,
    'Not Specified': 6
}

merged_data['formatted_experience_level'] = merged_data['formatted_experience_level'].map(experience_mapping)

#### Cleaning data in 'name' attribute

In [71]:
def clean_company_names(name):

    # Remove special characters, except spaces and dots
    name = re.sub(r'[^a-zA-Z0-9\w\s\.]', '', name)

    # Remove extra spaces
    name = re.sub(r'\s+', ' ', name).strip()

    # Handle hyphens and apostrophes
    name = name.replace('-', ' ')  # Replace hyphens with spaces
    name = name.replace('\'', '')  # Remove apostrophes
    name = re.sub(r'\s+', ' ', name)

    return name

merged_data['name'] = merged_data['name'].apply(clean_company_names)

#### Preprocessed Dataset

Shape of the cleaned data

In [72]:
merged_data_cleaned = merged_data
merged_data_cleaned.shape

(30338, 10)

Rename column names 

In [73]:

merged_data_cleaned = merged_data_cleaned.rename(columns={'name':'company','skill_name':'required_skill',
                                                           'formatted_experience_level':'experience_level'})

merged_data_cleaned = merged_data_cleaned.sort_values('job_id', ascending=False)



In [74]:
merged_data_cleaned['employee_count'] = merged_data_cleaned['employee_count'].astype(int)
merged_data_cleaned['follower_count'] = merged_data_cleaned['follower_count'].astype(int)

Display

In [75]:
merged_data_cleaned

,job_id,company_id,company,state,country,city,required_skill,experience_level,employee_count,follower_count
0,3757940104,553718,HearingLife,New Jersey,US,Somerset,Other,1,1171,11417
1,3757940025,2192142,Metalcraft of Mayville Inc.,WI,US,Mayville,"Management, Manufacturing",6,300,2923
2,3757938019,474443,U.S. Tsubaki Power Transmission LLC,Illinois,US,Wheeling,Engineering,6,314,8487
3,3757938018,18213359,Episcopal Communities Services,California,US,Altadena,"Management, Manufacturing",1,36,305
4,3757937095,437225,iHerb LLC,California,US,Irvine,Information Technology,3,1260,51651
...,...,...,...,...,...,...,...,...,...,...
32586,529257371,1244539,Ken Fulk Inc,California,US,San Francisco,"Design, Art/Creative, Information Technology",6,87,9790
32587,381055942,96654609,First Baptist Church Forney,Texas,US,Forney,Other,6,0,0
32588,133196985,1089558,Employvision Inc.,New Jersey,US,Somerset,"Accounting/Auditing, Finance",6,50,16176
32589,133114754,77766802,CargoLogin.,California,US,Santa Clarita,"Sales, Business Development",6,15,159


Save the preprocessed dataset to data_preprocessed.csv

In [79]:
merged_data_cleaned.to_csv('data_preprocessed/job_postings.csv', index=False)

### Prediction Model
#### Target: Predict employees future in the company based on the number of job postings that a company posted, skills and experience level they requried, and their employee count.

In [81]:
data = merged_data_cleaned

In [87]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
import lightgbm as lgb
import catboost as cb
import xgboost as xgb

# Creating a simulated target variable (job stability: 1 for stable, 0 for at risk)
np.random.seed(42)
data['job_stability'] = np.random.choice([0, 1], len(data), p=[0.2, 0.8])

# Encoding categorical variables
label_encoder = LabelEncoder()
data['required_skill_encoded'] = label_encoder.fit_transform(data['required_skill'])

# Selecting relevant features
features = ['required_skill_encoded', 'experience_level', 'employee_count', 'follower_count']

# Calculate correlation and covariance
correlation = data[features + ['job_stability']].corr()
covariance = data[features + ['job_stability']].cov()

# Print correlation and covariance
print("Correlation Matrix:\n", correlation)
print("\nCovariance Matrix:\n", covariance)

# Splitting the Data
X = data[features]
y = data['job_stability']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardizing the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Training the models
models = {
    'LightGBM': lgb.LGBMClassifier(random_state=42),
    'CatBoost': cb.CatBoostClassifier(verbose=0, random_state=42),
    'XGBoost': xgb.XGBClassifier(random_state=42)
}

results = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    acc = accuracy_score(y_test, pred)
    auc = roc_auc_score(y_test, pred)
    results[name] = {'Accuracy': acc}

# Display the results
for model, metrics in results.items():
    print(f"{model} - Accuracy: {metrics['Accuracy']}")

Correlation Matrix:
                         required_skill_encoded  experience_level  \
required_skill_encoded                1.000000          0.052080   
experience_level                      0.052080          1.000000   
employee_count                        0.041069          0.028513   
follower_count                        0.023655          0.065940   
job_stability                        -0.000321          0.000977   

                        employee_count  follower_count  job_stability  
required_skill_encoded        0.041069        0.023655      -0.000321  
experience_level              0.028513        0.065940       0.000977  
employee_count                1.000000        0.809043      -0.002515  
follower_count                0.809043        1.000000       0.002568  
job_stability                -0.002515        0.002568       1.000000  

Covariance Matrix:
                         required_skill_encoded  experience_level  \
required_skill_encoded            1.780841e+05   

In [83]:
data.to_csv('data_preprocessed/job_postings.csv', index=False)